# Parallel

In [1]:
import pandas as pd
import numpy as np
import random
import multiprocessing as mp

In [2]:
# constants

## Generate Data

## Parallelize w/ Multiprocessing - Singular Parameter

## Parallelize w/ Multiprocessing - Multiple Parameter

## Parallelize w/ Swifter

## Parallelize Sci-Kit Learn w/ Ray

## Concluding Remarks

---